In [2]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [3]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using AdaptiveTransportMap: transform!, evaluate, ncoeff, optimize, negative_log_likelihood!
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using Polynomials
using Optim: InverseDiagonal
using MLDataUtils
using Test

In [16]:
@testset "Verify log_pdf function" begin

  Nx = 2
  X  =  Matrix([0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]');
  B = MultiBasis(CstProHermite(6; scaled =true), Nx)

  idx = [0 0; 0 1; 1 0; 0 2; 2 0; 1 1]

  Nψ = 5

  coeff = [ -0.9905841755746164;
        0.6771992097558741;
       -2.243695806805015;
       -0.34591297359447354;
       -1.420159186008486;
       -0.5361337327704369]
  f = ExpandedFunction(B, idx, coeff)
  C = MapComponent(f)

  @test norm(log_pdf(C, X) - [  -1.572509004118956
                                -2.870725221050853
                                -1.285696671132943
                                -1.088115266085997
                                -2.396567575345843
                                -2.238446803642176
                                -5.999143198546611
                                -7.082679248037675])<1e-8
end

Test Summary:           | Pass  Total
Verify log_pdf function |    1      1


Test.DefaultTestSet("Verify log_pdf function", Any[], 1, false)

In [69]:
Nx = 2
m = 10
X  =  Matrix([0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]');

B = MultiBasis(CstProHermite(m; scaled =true), Nx);

M = HermiteMap(m, X);


idx1 = reshape([0; 1; 2;3],(4,1))
coeff1 =[-7.37835916713439;
         16.897974063168977;
         -0.20935914261414929;
          8.116032140446544];

M[1] = MapComponent(m, 1,  idx1, coeff1)

idx2 = [ 0  0; 0 1; 1 0; 2 0];
coeff2 = [2.995732886294909; -2.2624558703623903; -3.3791974895855486; 1.3808989978516617]

M[2] = MapComponent(m, 2,  idx2, coeff2);

In [72]:
@testset "Verify log_pdf function" begin
    # For diagonal rescaling
    Nx = 2
    m = 10
    X  =  Matrix([0.267333   1.43021;
              0.364979   0.607224;
             -1.23693    0.249277;
             -2.0526     0.915629;
             -0.182465   0.415874;
              0.412907   1.01672;
              1.41332   -0.918205;
              0.766647  -1.00445]');
    X0 = deepcopy(X)

    B = MultiBasis(CstProHermite(m; scaled =true), Nx);

    M = HermiteMap(m, X);


    idx1 = reshape([0; 1; 2;3],(4,1))
    coeff1 =[-7.37835916713439;
             16.897974063168977;
             -0.20935914261414929;
              8.116032140446544];

    M[1] = MapComponent(m, 1,  idx1, coeff1)

    idx2 = [ 0  0; 0 1; 1 0; 2 0];
    coeff2 = [2.995732886294909; -2.2624558703623903; -3.3791974895855486; 1.3808989978516617]

    M[2] = MapComponent(m, 2,  idx2, coeff2);

    @test norm(log_pdf(M, X) -   [-1.849644462509894;
                                  -1.239419791201162;
                                  -3.307440928766559;
                                  -2.482615159224171;
                                  -2.392017816948268;
                                  -1.331903266052277;
                                  -3.638709417555485;
                                  -2.242238529589172])<1e-6

    @test norm(X - X0)<1e-10
end

Test Summary:           | Pass  Total
Verify log_pdf function |    2      2


Test.DefaultTestSet("Verify log_pdf function", Any[], 2, false)

In [80]:
# For Cholesky refactorization

Nx = 2
m = 10
X  =  Matrix([0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]');
X0 = deepcopy(X)

B = MultiBasis(CstProHermite(m; scaled =true), Nx);

M = HermiteMap(m, X; diag = false);


idx1 = reshape([0; 1; 2;3],(4,1))
coeff1 =[-7.37835916713439;
         16.897974063168977;
         -0.20935914261414929;
          8.116032140446544];

M[1] = MapComponent(m, 1,  idx1, coeff1)

idx2 = [ 0  0; 0 1; 1 0; 2 0];
coeff2 = [2.995732886294909; -2.2624558703623903; -3.3791974895855486; 1.3808989978516617]

M[2] = MapComponent(m, 2,  idx2, coeff2);

log_pdf(M, X)

8-element Array{Float64,1}:
 -2.2844206901938313
 -1.1136936437930662
 -3.927922581187449
 -2.461816758104064
 -2.328286973844927
 -1.5299960111650504
 -3.444940697348022
 -1.9994193352667469

In [ ]:
#     @test norm(log_pdf(M, X) -   [-1.849644462509894;
#                                   -1.239419791201162;
#                                   -3.307440928766559;
#                                   -2.482615159224171;
#                                   -2.392017816948268;
#                                   -1.331903266052277;
#                                   -3.638709417555485;
#                                   -2.242238529589172])<1e-6

#     @test norm(X - X0)<1e-10